In [72]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import statsmodels.api as sm
import dash
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.svm import SVR
from dash.dependencies import Input, Output, State
from dash import dcc, html

Wczytanie danych

In [2]:
apps = pd.read_csv('appearances.csv')
#club_games = pd.read_csv('club_games.csv')
#clubs = pd.read_csv('clubs.csv')
#competitions = pd.read_csv('competitions.csv')
#game_events = pd.read_csv('game_events.csv')
#game_lineups = pd.read_csv('game_lineups.csv')
#games = pd.read_csv('games.csv')
player_val = pd.read_csv('player_valuations.csv')
players = pd.read_csv('players.csv')

In [3]:
csv = [apps, player_val, players]

In [4]:
apps

,appearance_id,game_id,player_id,player_club_id,player_current_club_id,date,player_name,competition_id,yellow_cards,red_cards,goals,assists,minutes_played
0,2231978_38004,2231978,38004,853,235,2012-07-03,Aurélien Joachim,CLQ,0,0,2,0,90
1,2233748_79232,2233748,79232,8841,2698,2012-07-05,Ruslan Abyshov,ELQ,0,0,0,0,90
2,2234413_42792,2234413,42792,6251,465,2012-07-05,Sander Puri,ELQ,0,0,0,0,45
3,2234418_73333,2234418,73333,1274,6646,2012-07-05,Vegar Hedenstad,ELQ,0,0,0,0,90
4,2234421_122011,2234421,122011,195,3008,2012-07-05,Markus Henriksen,ELQ,0,0,0,1,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1573074,4331989_607854,4331989,607854,800,800,2024-05-02,Éderson,EL,0,0,0,0,90
1573075,4331989_616631,4331989,616631,800,800,2024-05-02,Matteo Ruggeri,EL,0,0,0,0,90
1573076,4331989_623570,4331989,623570,244,244,2024-05-02,Iliman Ndiaye,EL,0,0,0,0,18
1573077,4331989_691316,4331989,691316,244,244,2024-05-02,Luis Henrique,EL,0,0,0,0,90


<h4>Wstępny przegląd danych. UWAGA! Dane na późniejszym etapie są sprawdzane pod kątem ich czystości (np występowania NaN), ponieważ tworzone są nowe dataframe'y.</h4>

In [5]:
print(apps.isna().any())
apps = apps.dropna()
print(apps.isna().count())


appearance_id             False
game_id                   False
player_id                 False
player_club_id            False
player_current_club_id    False
date                      False
player_name                True
competition_id            False
yellow_cards              False
red_cards                 False
goals                     False
assists                   False
minutes_played            False
dtype: bool
appearance_id             1573073
game_id                   1573073
player_id                 1573073
player_club_id            1573073
player_current_club_id    1573073
date                      1573073
player_name               1573073
competition_id            1573073
yellow_cards              1573073
red_cards                 1573073
goals                     1573073
assists                   1573073
minutes_played            1573073
dtype: int64


In [6]:
for file in csv:
    print(file.isna().any())

appearance_id             False
game_id                   False
player_id                 False
player_club_id            False
player_current_club_id    False
date                      False
player_name                True
competition_id            False
yellow_cards              False
red_cards                 False
goals                     False
assists                   False
minutes_played            False
dtype: bool
player_id                              False
date                                   False
market_value_in_eur                    False
current_club_id                        False
player_club_domestic_competition_id    False
dtype: bool
player_id                               False
first_name                               True
last_name                               False
name                                    False
last_season                             False
current_club_id                         False
player_code                             False
country_of_bi

In [7]:
players.columns

Index(['player_id', 'first_name', 'last_name', 'name', 'last_season',
       'current_club_id', 'player_code', 'country_of_birth', 'city_of_birth',
       'country_of_citizenship', 'date_of_birth', 'sub_position', 'position',
       'foot', 'height_in_cm', 'contract_expiration_date', 'agent_name',
       'image_url', 'url', 'current_club_domestic_competition_id',
       'current_club_name', 'market_value_in_eur',
       'highest_market_value_in_eur'],
      dtype='object')

In [8]:
players.isna().any()

player_id                               False
first_name                               True
last_name                               False
name                                    False
last_season                             False
current_club_id                         False
player_code                             False
country_of_birth                         True
city_of_birth                            True
country_of_citizenship                   True
date_of_birth                            True
sub_position                             True
position                                False
foot                                     True
height_in_cm                             True
contract_expiration_date                 True
agent_name                               True
image_url                               False
url                                     False
current_club_domestic_competition_id    False
current_club_name                       False
market_value_in_eur               

In [9]:
player_val.columns

Index(['player_id', 'date', 'market_value_in_eur', 'current_club_id',
       'player_club_domestic_competition_id'],
      dtype='object')

<h4>Statystyki klubów Premier League</h4>

In [10]:
premier_league_clubs = ['Tottenham Hotspur Football Club', 'Association Football Club Bournemouth', 'Liverpool Football Club', 'Brighton and Hove Albion Football Club',
                       'Nottingham Forest Football Club', 'Luton Town Football Club', 'Newcastle United Football Club', 'Brentford Football Club', 'Crystal Palace Football Club',
                       'West Ham United Football Club', 'Fulham Football Club', 'Manchester United Football Club', 'Burnley Football Club', 'Arsenal Football Club', 
                        'Manchester City Football Club', 'Aston Villa Football Club', 'Wolverhampton Wanderers Football Club', 'Sheffield United Football Club', 
                        'Chelsea Football Club', 'Everton Football Club']
premier_league_players = players[(players['current_club_name'].isin(premier_league_clubs)) & (players['last_season'] == 2023)]
club_value_premier_league = premier_league_players.groupby('current_club_name')['market_value_in_eur'].sum().reset_index()
club_value_premier_league.columns = ['Club', 'Squad Market Value (EUR)']
club_value_premier_league = club_value_premier_league.sort_values(by='Squad Market Value (EUR)', ascending = False)
club_value_premier_league['Market Value'] = club_value_premier_league['Squad Market Value (EUR)'].apply(lambda x: '{:,.2f} EUR'.format(x))
fig = go.Figure(data=[go.Pie(labels=club_value_premier_league['Club'], 
                             values=club_value_premier_league['Squad Market Value (EUR)'])])

fig.update_layout(title='Market value of Premier League clubs')

fig.show()

<h4>Statystyki klubów Serie A</h4>

In [11]:
serie_a_clubs = ["Associazione Sportiva Roma", "Bologna Football Club 1909", "Udinese Calcio", "Football Club Internazionale Milano S.p.A.", "Società Sportiva Calcio Napoli",
                 "Verona Hellas Football Club", "U.S. Salernitana 1919 S.r.l.", "Juventus Football Club", "Associazione Calcio Monza", "Società Sportiva Lazio S.p.A.",
                 "Unione Sportiva Sassuolo Calcio", "Cagliari Calcio", "Associazione Calcio Fiorentina", "Associazione Calcio Milan", "Genoa Cricket and Football Club",
                 "Frosinone Calcio S.r.l.", "Unione Sportiva Lecce", "Torino Calcio", "Empoli Football Club S.r.l.", "Atalanta Bergamasca Calcio S.p.a."]
serie_a_players = players[(players['current_club_name'].isin(serie_a_clubs)) & (players['last_season'] == 2023)]
club_value_serie_a = serie_a_players.groupby('current_club_name')['market_value_in_eur'].sum().reset_index()
club_value_serie_a.columns = ['Club', 'Squad Market Value (EUR)']
club_value_serie_a = club_value_serie_a.sort_values(by='Squad Market Value (EUR)', ascending = False)
club_value_serie_a['Market Value'] = club_value_serie_a['Squad Market Value (EUR)'].apply(lambda x: '{:,.2f} EUR'.format(x))
fig = go.Figure(data=[go.Pie(labels=club_value_serie_a['Club'], 
                             values=club_value_serie_a['Squad Market Value (EUR)'])])

fig.update_layout(title='Market value of Serie A clubs')

fig.show()

<h4>Statystyki klubów La Liga 

In [12]:
la_liga_clubs = ['Deportivo Alavés S.A.D.', 'Club Atlético de Madrid S.A.D.', 'Futbol Club Barcelona', 'Villarreal Club de Fútbol S.A.D.', 'Villarreal Club de Fútbol S.A.D.',
                'Club Atlético Osasuna', 'Getafe Club de Fútbol S.A.D. Team Dubai', 'Real Club Deportivo Mallorca S.A.D.', 'Real Sociedad de Fútbol S.A.D.', 'Rayo Vallecano de Madrid S.A.D.',
                'Athletic Club Bilbao', 'Real Betis Balompié S.A.D.', 'Cádiz Club de Fútbol S.A.D', 'Real Madrid Club de Fútbol', 'Real Club Celta de Vigo S. A. D.', 'Granada Club de Fútbol S.A.D.',
                'Unión Deportiva Almería S.A.D.', 'Girona Fútbol Club S. A. D.', 'Unión Deportiva Las Palmas S.A.D.', 'Sevilla Fútbol Club S.A.D.', 'Valencia Club de Fútbol S. A. D.']
la_liga_players = players[(players['current_club_name'].isin(la_liga_clubs)) & (players['last_season'] == 2023)]
club_value_la_liga = la_liga_players.groupby('current_club_name')['market_value_in_eur'].sum().reset_index()
club_value_la_liga.columns = ['Club', 'Squad Market Value (EUR)']
club_value_la_liga = club_value_la_liga.sort_values(by='Squad Market Value (EUR)', ascending = False)
club_value_la_liga['Market Value'] = club_value_la_liga['Squad Market Value (EUR)'].apply(lambda x: '{:,.2f} EUR'.format(x))
fig = go.Figure(data=[go.Pie(labels=club_value_la_liga['Club'], 
                             values=club_value_la_liga['Squad Market Value (EUR)'])])

fig.update_layout(title='Market value of La Liga clubs')

fig.show()

<h4>Statystyki klubów Bundesligi</h4>

In [13]:
bundesliga_clubs = ['Sportverein Darmstadt 1898 e. V.', 'Borussia Verein für Leibesübungen 1900 Mönchengladbach', 'Verein für Leibesübungen Wolfsburg',
 'Bayer 04 Leverkusen Fußball', 'Eintracht Frankfurt Fußball AG', 'Borussia Dortmund', 'Verein für Bewegungsspiele Stuttgart 1893',
 'Sportverein Werder Bremen von 1899', 'RasenBallsport Leipzig', '1. FC Union Berlin', 'FC Augsburg 1907', '1. Fußball-Club Köln', '1. Fußball- und Sportverein Mainz 05', 'Sport-Club Freiburg',
 'FC Schalke 04', 'Hertha BSC', 'FC Bayern München', 'TSG 1899 Hoffenheim Fußball-Spielbetriebs GmbH']
bundesliga_players = players[(players['current_club_name'].isin(bundesliga_clubs)) & (players['last_season'] == 2023)]
club_value_bundesliga = bundesliga_players.groupby('current_club_name')['market_value_in_eur'].sum().reset_index()
club_value_bundesliga.columns = ['Club', 'Squad Market Value (EUR)']
club_value_bundesliga = club_value_bundesliga.sort_values(by='Squad Market Value (EUR)', ascending = False)
club_value_bundesliga['Market Value'] = club_value_bundesliga['Squad Market Value (EUR)'].apply(lambda x: '{:,.2f} EUR'.format(x))
fig = go.Figure(data=[go.Pie(labels=club_value_bundesliga['Club'], 
                             values=club_value_bundesliga['Squad Market Value (EUR)'])])

fig.update_layout(title='Market value of Bundesliga clubs')

fig.show()


<h4>Statystyki klubów Ligue 1</h4>

In [14]:
ligue_clubs = ['Montpellier Hérault Sport Club', 'Lille Olympique Sporting Club Lille Métropole', 'Football Club Lorient-Bretagne Sud', 'Stade Rennais Football Club',
 'Football Club de Nantes', 'Stade de Reims', 'Association sportive de Monaco Football Club', 'FC Sochaux-Montbéliard', 'Olympique Lyonnais', 'Racing Club de Strasbourg Alsace',
 'Olympique de Marseille', 'Paris Saint-Germain Football Club', 'Clermont Foot 63', 'Le Havre Athletic Club', 'Racing Club de Lens', 'Toulouse Football Club',
 "Olympique Gymnaste Club Nice Côte d'Azur", 'Football Club de Metz']
ligue_players = players[(players['current_club_name'].isin(ligue_clubs)) & (players['last_season'] == 2023)]
club_value_ligue = ligue_players.groupby('current_club_name')['market_value_in_eur'].sum().reset_index()
club_value_ligue.columns = ['Club', 'Squad Market Value (EUR)']
club_value_ligue = club_value_ligue.sort_values(by='Squad Market Value (EUR)', ascending = False)
club_value_ligue['Market Value'] = club_value_ligue['Squad Market Value (EUR)'].apply(lambda x: '{:,.2f} EUR'.format(x))
fig = go.Figure(data=[go.Pie(labels=club_value_ligue['Club'], 
                             values=club_value_ligue['Squad Market Value (EUR)'])])

fig.update_layout(title='Market value of Ligue 1 clubs')

fig.show()

In [15]:
top_leagues = ['CL', 'EL', 'FR1', 'GB1', 'ES1', 'IT1', 'L1']

In [16]:
top5_leagues = [ 'FR1', 'GB1', 'ES1', 'IT1', 'L1']

In [17]:
top5_id = apps[apps['competition_id'].isin(top_leagues)]['player_id']

Filtr na piłkarzy grających w Premier League oraz będących obywatelami Anglii + tych, którzy tylko grali w PL

In [18]:
filt = players[(players['country_of_citizenship'] == 'England') & (players['current_club_domestic_competition_id'] == 'GB1')]
english_players = filt['player_id']
top5_players = players[players['player_id'].isin(top5_id)]['player_id']
top5_players

0             10
1             26
2             65
3             77
4             80
          ...   
30477    1144999
30478    1146029
30484    1159022
30499    1176345
30502    1178474
Name: player_id, Length: 11411, dtype: int64

Piłkarze posortowani po wartości w czasie (wyszukani za pomocą filtra)

In [19]:

filt2 = player_val[player_val['player_id'].isin(english_players.values)]
filt2_sorted = filt2.sort_values(by='player_id')
filt3 = player_val[player_val['player_id'].isin(top5_players.values)]
filt3_sorted = filt3.sort_values(by='player_id')

Statystyki graczy pasujących do filtru (eng + gb1)

In [20]:
start_date = '2012-08-31'
end_date = '2013-06-14'
apps['date'] = pd.to_datetime(apps['date'])

start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)
seasons = []

for x in range(11):
    season_filter = apps[(apps['date'] >= start_date) & (apps['date'] <= end_date) & (apps['competition_id'] == 'GB1') & (apps['player_id'].isin(english_players))]
    goals_dict = season_filter.groupby('player_id')['goals'].sum().to_dict()
    yellow_cards_dict = season_filter.groupby('player_id')['yellow_cards'].sum().to_dict()
    red_cards_dict = season_filter.groupby('player_id')['red_cards'].sum().to_dict()
    assists_dict = season_filter.groupby('player_id')['assists'].sum().to_dict()
    minutes_played_dict = season_filter.groupby('player_id')['minutes_played'].sum().to_dict()

    df_goals = pd.DataFrame(list(goals_dict.items()), columns=['player_id', 'goals'])
    df_assits = pd.DataFrame(list(assists_dict.items()), columns=['player_id', 'assists'])
    df_yellow = pd.DataFrame(list(yellow_cards_dict.items()), columns=['player_id', 'yellow_cards'])
    df_red = pd.DataFrame(list(red_cards_dict.items()), columns=['player_id', 'red_cards'])
    df_minutes = pd.DataFrame(list(minutes_played_dict.items()), columns=['player_id', 'minutes_played'])

    season = pd.merge(df_goals, df_assits, on='player_id', how='outer')
    season = pd.merge(season, df_yellow, on='player_id', how='outer')
    season = pd.merge(season, df_red, on='player_id', how='outer')
    season = pd.merge(season, df_minutes, on='player_id', how='outer')

    seasons.append(season)

    start_date += pd.DateOffset(years=1)
    end_date += pd.DateOffset(years=1)

C:\Users\Max\AppData\Local\Temp\ipykernel_29544\3039982494.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Statystyki wszystkich graczy (top5 lig)

In [21]:
start_date = '2012-08-31'
end_date = '2013-06-14'
apps['date'] = pd.to_datetime(apps['date'])

start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)
seasons_all_players = []

for x in range(11):
    season_filter = apps[(apps['date'] >= start_date) & (apps['date'] <= end_date) & (apps['competition_id'] == 'GB1') & (apps['player_id'].isin(top5_players))]
    goals_dict = season_filter.groupby('player_id')['goals'].sum().to_dict()
    yellow_cards_dict = season_filter.groupby('player_id')['yellow_cards'].sum().to_dict()
    red_cards_dict = season_filter.groupby('player_id')['red_cards'].sum().to_dict()
    assists_dict = season_filter.groupby('player_id')['assists'].sum().to_dict()
    minutes_played_dict = season_filter.groupby('player_id')['minutes_played'].sum().to_dict()

    df_goals = pd.DataFrame(list(goals_dict.items()), columns=['player_id', 'goals'])
    df_assits = pd.DataFrame(list(assists_dict.items()), columns=['player_id', 'assists'])
    df_yellow = pd.DataFrame(list(yellow_cards_dict.items()), columns=['player_id', 'yellow_cards'])
    df_red = pd.DataFrame(list(red_cards_dict.items()), columns=['player_id', 'red_cards'])
    df_minutes = pd.DataFrame(list(minutes_played_dict.items()), columns=['player_id', 'minutes_played'])

    season = pd.merge(df_goals, df_assits, on='player_id', how='outer')
    season = pd.merge(season, df_yellow, on='player_id', how='outer')
    season = pd.merge(season, df_red, on='player_id', how='outer')
    season = pd.merge(season, df_minutes, on='player_id', how='outer')

    seasons_all_players.append(season)

    start_date += pd.DateOffset(years=1)
    end_date += pd.DateOffset(years=1)

C:\Users\Max\AppData\Local\Temp\ipykernel_29544\3938328921.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Dodanie imion dla graczy (top5 lig)

In [22]:
names_all_players = players[['name', 'player_id']][players['player_id'].isin(top5_players.values)]
merged_all_players = filt3_sorted.merge(names_all_players, on='player_id')
merged_all_players

,player_id,date,market_value_in_eur,current_club_id,player_club_domestic_competition_id,name
0,10,2012-07-03,6000000,398,IT1,Miroslav Klose
1,10,2015-01-07,1000000,398,IT1,Miroslav Klose
2,10,2005-05-05,12000000,398,IT1,Miroslav Klose
3,10,2016-01-04,1000000,398,IT1,Miroslav Klose
4,10,2015-07-01,1000000,398,IT1,Miroslav Klose
...,...,...,...,...,...,...
247690,1159022,2024-03-12,300000,8970,IT1,Mateus Lusuardi
247691,1176345,2024-03-19,500000,273,FR1,Mahamadou Nagida
247692,1176345,2023-12-18,150000,273,FR1,Mahamadou Nagida
247693,1178474,2024-03-19,400000,1421,FR1,Amadou Koné


Wycena graczy (top5 lig)

In [23]:
start_date = '2012-08-31'
end_date = '2013-06-14'
merged_all_players['date'] = pd.to_datetime(merged_all_players['date'])
start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)
all_values = []

for i in range(11):

    name_val = merged_all_players[(merged_all_players['date'] >= start_date) & (merged_all_players['date'] <= end_date)]

    all_values.append(name_val)

    temp = all_values[i]['player_id'].drop_duplicates(keep='first')
    all_values[i]['player_id'] = temp
    all_values[i] = all_values[i].dropna()
    #all_values[i] = all_values[i].drop(columns='name')
    start_date += pd.DateOffset(years=1)
    end_date += pd.DateOffset(years=1)

C:\Users\Max\AppData\Local\Temp\ipykernel_29544\279105413.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Połączenie wyceny ze statystykami

In [24]:
for i in range(len(seasons_all_players)):
    seasons_all_players[i] = pd.merge(seasons_all_players[i], all_values[i], on='player_id', how='outer')
    seasons_all_players[i] = seasons_all_players[i].dropna()



Sezony od 2012 do 2022

In [25]:
season_stats = {
    'season_2012_2013': seasons[0],
    'season_2013_2014': seasons[1],
    'season_2014_2015': seasons[2],
    'season_2015_2016': seasons[3],
    'season_2016_2017': seasons[4],
    'season_2017_2018': seasons[5],
    'season_2018_2019': seasons[6],
    'season_2019_2020': seasons[7],
    'season_2020_2021': seasons[8],
    'season_2021_2022': seasons[9],
    'season_2022_2023': seasons[10]
}

In [26]:
season_stats_all_players = {
    'season_2012_2013': seasons_all_players[0],
    'season_2013_2014': seasons_all_players[1],
    'season_2014_2015': seasons_all_players[2],
    'season_2015_2016': seasons_all_players[3],
    'season_2016_2017': seasons_all_players[4],
    'season_2017_2018': seasons_all_players[5],
    'season_2018_2019': seasons_all_players[6],
    'season_2019_2020': seasons_all_players[7],
    'season_2020_2021': seasons_all_players[8],
    'season_2021_2022': seasons_all_players[9],
    'season_2022_2023': seasons_all_players[10]
}

In [27]:
for i, season in enumerate(season_stats):
    print(season)

season_2012_2013
season_2013_2014
season_2014_2015
season_2015_2016
season_2016_2017
season_2017_2018
season_2018_2019
season_2019_2020
season_2020_2021
season_2021_2022
season_2022_2023


Dodanie imion do ramki (eng + gb1)

In [28]:
names = players[['name', 'player_id']][players['player_id'].isin(english_players.values)]
merged = filt2_sorted.merge(names, on='player_id')
merged

,player_id,date,market_value_in_eur,current_club_id,player_club_domestic_competition_id,name
0,1397,2004-10-04,30000000,512,GB1,Michael Owen
1,1397,2011-08-08,5000000,512,GB1,Michael Owen
2,1397,2011-02-01,7000000,512,GB1,Michael Owen
3,1397,2007-05-03,15000000,512,GB1,Michael Owen
4,1397,2012-02-06,2500000,512,GB1,Michael Owen
...,...,...,...,...,...,...
10981,922769,2023-10-09,2500000,762,GB1,Lewis Miley
10982,922769,2023-12-19,13000000,762,GB1,Lewis Miley
10983,933017,2022-11-03,250000,703,GB1,Zach Abbott
10984,936901,2022-11-03,100000,989,GB1,Dominic Sadi


In [29]:
for i in range(len(seasons)):
    seasons[i] = seasons[i].merge(names, on='player_id')


Dodanie wartosci pilkarzy jesli data aktualizacji ceny miesci sie w granicach sezonu.

In [30]:
start_date = '2012-08-31'
end_date = '2013-06-14'
merged['date'] = pd.to_datetime(merged['date'])
start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)
values = []

for i in range(11):

    name_val = merged[(merged['date'] >= start_date) & (merged['date'] <= end_date)]
    values.append(name_val)

    temp = values[i]['player_id'].drop_duplicates(keep='first')
    values[i]['player_id'] = temp
    values[i] = values[i].dropna()
    values[i] = values[i].drop(columns='name')
    start_date += pd.DateOffset(years=1)
    end_date += pd.DateOffset(years=1)

C:\Users\Max\AppData\Local\Temp\ipykernel_29544\91763524.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Max\AppData\Local\Temp\ipykernel_29544\91763524.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Max\AppData\Local\Temp\ipykernel_29544\91763524.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [31]:
for i in range(len(seasons)):
    seasons[i] = pd.merge(seasons[i], values[i], on='player_id', how='outer')
    seasons[i] = seasons[i].dropna()

In [32]:
# start_date = '2012-08-31'
# end_date = '2013-06-14'
# date = merged_all_players['date']
# date = pd.to_datetime(date)

# start_date = pd.to_datetime(start_date)
# end_date = pd.to_datetime(end_date)
# prices = {}

# for y in range(len(seasons_all_players)):
#     for x in range(len(merged_all_players['date'])):
#         if ((date.iloc[x] >= start_date) & (date.iloc[x] <= end_date)):
#             prices[merged_all_players['player_id'].iloc[x]] = merged_all_players['market_value_in_eur'].iloc[x]
#             df_prices = pd.DataFrame(list(prices.items()), columns=['player_id', 'market_value_in_eur'])
#     seasons_all_players[y] = seasons_all_players[y].merge(df_prices, on='player_id', how='outer')        
#     start_date += pd.DateOffset(years=1)
#     end_date += pd.DateOffset(years=1)

In [33]:
# for season in seasons_all_players:
#     season.dropna(inplace=True)

In [34]:
for season in seasons:
    season.dropna(inplace=True)

for season in seasons_all_players:
    season.dropna(inplace=True)

In [35]:
seasons_all_players[0].columns

Index(['player_id', 'goals', 'assists', 'yellow_cards', 'red_cards',
       'minutes_played', 'date', 'market_value_in_eur', 'current_club_id',
       'player_club_domestic_competition_id', 'name'],
      dtype='object')

In [36]:
seasons[0]

,player_id,goals,assists,yellow_cards,red_cards,minutes_played,name,date,market_value_in_eur,current_club_id,player_club_domestic_competition_id
0,1397.0,1.0,0.0,0.0,0.0,94.0,Michael Owen,2013-01-28,1000000.0,512.0,GB1
1,3109.0,10.0,8.0,5.0,0.0,3042.0,Steven Gerrard,2013-01-28,11500000.0,31.0,GB1
2,3118.0,0.0,1.0,1.0,0.0,196.0,Kieron Dyer,2013-03-02,500000.0,1039.0,GB1
4,3130.0,0.0,0.0,3.0,0.0,1181.0,Titus Bramble,2013-01-28,2000000.0,289.0,GB1
6,3160.0,4.0,0.0,1.0,0.0,769.0,John Terry,2013-01-28,12000000.0,631.0,GB1
...,...,...,...,...,...,...,...,...,...,...,...
233,128912.0,0.0,1.0,0.0,0.0,1015.0,Andre Wisdom,2012-11-06,500000.0,31.0,GB1
240,134424.0,1.0,0.0,0.0,0.0,30.0,Nick Powell,2013-01-28,5000000.0,985.0,GB1
241,134425.0,2.0,6.0,1.0,0.0,1658.0,Raheem Sterling,2013-01-28,11500000.0,631.0,GB1
250,181579.0,0.0,1.0,0.0,0.0,403.0,James Ward-Prowse,2012-09-28,500000.0,379.0,GB1


In [37]:
def value_goals(name):
    fig = px.scatter()

    for i in range(len(seasons)):
        fig.add_scatter(x=seasons[i]['goals'][seasons[i]['name'] == name], y=seasons[i]['market_value_in_eur'][seasons[i]['name'] == name], mode='markers', name=f'Season {2012 + i}/{2013+i}')

    fig.update_layout(title=f'Market value of {name}',
                  xaxis_title='Goals',
                  yaxis_title='Market value in eur')

    fig.show()

In [38]:
value_goals('Marcus Rashford')


In [39]:
def stats(name):
    

    data = []
    for i, season in enumerate(seasons):
        player_data = season[season['name'] == name]
        if not player_data.empty:
            season_data = {
                'season': f'Season {2012 + i}/{2013 + i}',
                'goals': player_data['goals'].values[0],
                'assists': player_data['assists'].values[0],
                'yellow_cards': player_data['yellow_cards'].values[0],
                'red_cards': player_data['red_cards'].values[0],
                'market_value_in_eur': player_data['market_value_in_eur'].values[0] / 1000000
            }
            data.append(season_data)
    df = pd.DataFrame(data)
    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=df['season'],
        y=df['goals'],
        name='Goals'
    ))

    fig.add_trace(go.Bar(
        x=df['season'],
        y=df['assists'],
        name='Assists'
    ))

    fig.add_trace(go.Bar(
        x=df['season'],
        y=df['yellow_cards'],
        name='Yellow Cards'
    ))

    fig.add_trace(go.Bar(
        x=df['season'],
        y=df['red_cards'],
        name='Red Cards'
    ))

    fig.add_trace(go.Bar(
        x=df['season'],
        y=df['market_value_in_eur'],
        name='Value'
    ))

    fig.update_layout(
        title=f'Statistics of {name} Over Seasons',
        xaxis_title='Season',
        yaxis_title='Count',
        barmode='group',
        legend_title='Statistics'
    )

    fig.show()


In [40]:
stats('Marcus Rashford')

In [41]:
season_stats = {
    'season_2012_2013': seasons[0],
    'season_2013_2014': seasons[1],
    'season_2014_2015': seasons[2],
    'season_2015_2016': seasons[3],
    'season_2016_2017': seasons[4],
    'season_2017_2018': seasons[5],
    'season_2018_2019': seasons[6],
    'season_2019_2020': seasons[7],
    'season_2020_2021': seasons[8],
    'season_2021_2022': seasons[9],
    'season_2022_2023': seasons[10]
}

In [42]:
season_stats_all_players = {
    'season_2012_2013': seasons_all_players[0],
    'season_2013_2014': seasons_all_players[1],
    'season_2014_2015': seasons_all_players[2],
    'season_2015_2016': seasons_all_players[3],
    'season_2016_2017': seasons_all_players[4],
    'season_2017_2018': seasons_all_players[5],
    'season_2018_2019': seasons_all_players[6],
    'season_2019_2020': seasons_all_players[7],
    'season_2020_2021': seasons_all_players[8],
    'season_2021_2022': seasons_all_players[9],
    'season_2022_2023': seasons_all_players[10]
}

In [43]:
encoder = LabelEncoder()

info = players[['player_id', 'date_of_birth', 'sub_position', 'position', 'height_in_cm']][players['player_id'].isin(english_players.values)]
info_all = players[['player_id', 'date_of_birth', 'sub_position', 'position', 'height_in_cm']][players['player_id'].isin(top5_players.values)]
info['sub_position_encoded'] = encoder.fit_transform(info['sub_position'])
info_all['sub_position_encoded'] = encoder.fit_transform(info_all['sub_position'])
info['position_encoded'] = encoder.fit_transform(info['position'])
info_all['position_encoded'] = encoder.fit_transform(info_all['position'])

info.dropna(inplace=True)
info_all.dropna(inplace=True)
info

,player_id,date_of_birth,sub_position,position,height_in_cm,sub_position_encoded,position_encoded
63,1397,1979-12-14,Centre-Forward,Attack,173.0,3,0
138,3109,1980-05-30,Central Midfield,Midfield,183.0,1,3
141,3118,1978-12-29,Right Midfield,Midfield,170.0,9,3
142,3120,1977-11-05,Goalkeeper,Goalkeeper,190.0,5,2
143,3130,1981-07-31,Centre-Back,Defender,192.0,2,1
...,...,...,...,...,...,...,...
29527,887433,2006-01-19,Left Winger,Attack,176.0,7,0
29663,907215,2005-08-17,Defensive Midfield,Midfield,175.0,4,3
29767,922769,2006-05-01,Central Midfield,Midfield,189.0,1,3
30241,1031256,2003-09-08,Centre-Forward,Attack,185.0,3,0


In [44]:
year_counter = -12

for i in range(len(seasons)):
    seasons[i] = seasons[i].merge(info, on='player_id', how='outer')
    seasons[i] = seasons[i].dropna()
    seasons[i].rename(columns={'date_of_birth' : 'age'}, inplace=True)
    seasons[i]['age'] = (datetime.now().year - pd.to_datetime(seasons[i]['age']).dt.year) + year_counter
    year_counter += 1

In [45]:
year_counter = -12

for i in range(len(seasons_all_players)):
    seasons_all_players[i] = seasons_all_players[i].merge(info_all, on='player_id', how='outer')
    seasons_all_players[i] = seasons_all_players[i].dropna()
    seasons_all_players[i].rename(columns={'date_of_birth' : 'age'}, inplace=True)
    seasons_all_players[i]['age'] = (datetime.now().year - pd.to_datetime(seasons_all_players[i]['age']).dt.year) + year_counter
    year_counter += 1

In [46]:
for i in range(len(seasons)):
    seasons[i] = seasons[i].drop(columns=['date','current_club_id','player_club_domestic_competition_id','sub_position','position'])
    seasons_all_players[i] = seasons_all_players[i].drop(columns=['date','current_club_id','player_club_domestic_competition_id','sub_position','position'])

In [47]:
season_stats = {
    'season_2012_2013': seasons[0],
    'season_2013_2014': seasons[1],
    'season_2014_2015': seasons[2],
    'season_2015_2016': seasons[3],
    'season_2016_2017': seasons[4],
    'season_2017_2018': seasons[5],
    'season_2018_2019': seasons[6],
    'season_2019_2020': seasons[7],
    'season_2020_2021': seasons[8],
    'season_2021_2022': seasons[9],
    'season_2022_2023': seasons[10]
}

season_stats_all_players = {
    'season_2012_2013': seasons_all_players[0],
    'season_2013_2014': seasons_all_players[1],
    'season_2014_2015': seasons_all_players[2],
    'season_2015_2016': seasons_all_players[3],
    'season_2016_2017': seasons_all_players[4],
    'season_2017_2018': seasons_all_players[5],
    'season_2018_2019': seasons_all_players[6],
    'season_2019_2020': seasons_all_players[7],
    'season_2020_2021': seasons_all_players[8],
    'season_2021_2022': seasons_all_players[9],
    'season_2022_2023': seasons_all_players[10]
}

In [48]:
combined_stats = pd.concat(season_stats.values(), keys=season_stats.keys())
combined_stats_all = pd.concat(season_stats_all_players.values(), keys=season_stats_all_players.keys())

In [49]:
combined_stats_all

player_id  goals  assists  yellow_cards  red_cards  \
season_2012_2013 4          132.0    2.0      1.0           2.0        0.0   
                 21        1094.0    0.0      0.0           2.0        0.0   
                 27        1327.0    0.0      0.0           1.0        0.0   
                 31        1397.0    1.0      0.0           0.0        0.0   
                 69        2953.0    0.0      1.0           6.0        0.0   
...                           ...    ...      ...           ...        ...   
season_2022_2023 10944   820374.0    0.0      0.0           0.0        0.0   
                 10950   823231.0    1.0      3.0           1.0        0.0   
                 10962   834742.0    0.0      0.0           0.0        0.0   
                 11005   864799.0    1.0      0.0           2.0        0.0   
                 11126   983989.0    1.0      1.0           2.0        0.0   

                        minutes_played  market_value_in_eur  \
season_2012_2013 4               612.0            2000000.0   
                 21              149.0            2500000.0   
                 27              118.0            1250000.0   
                 31               94.0            1000000.0   
                 69             1260.0            1500000.0   
...                                ...                  ...   
season_2022_2023 10944            10.0             300000.0   
                 10950          1245.0           10000000.0   
                 10962            24.0             900000.0   
                 11005           508.0           13000000.0   
                 11126           562.0            6000000.0   

                                      name  age  height_in_cm  \
season_2012_2013 4           Tomas Rosicky   32         179.0   
                 21       Emanuel Pogatetz   29         191.0   
                 27            Alou Diarra   31         189.0   
                 31           Michael Owen   33         173.0   
                 69          Paul Scharner   32         191.0   
...                                    ...  ...           ...   
season_2022_2023 10944       Kobbie Mainoo   17         175.0   
                 10950      Dango Ouattara   20         177.0   
                 10962        Mateo Joseph   19         180.0   
                 11005     Stefan Bajcetic   18         185.0   
                 11126  Facundo Buonanotte   18         174.0   

                        sub_position_encoded  position_encoded  
season_2012_2013 4                       0.0               3.0  
                 21                      2.0               1.0  
                 27                      4.0               3.0  
                 31                      3.0               0.0  
                 69                      4.0               3.0  
...                                      ...               ...  
season_2022_2023 10944                   1.0               3.0  
                 10950                  10.0               0.0  
                 10962                   3.0               0.0  
                 11005                   4.0               3.0  
                 11126                   0.0               3.0  

[5413 rows x 12 columns]

In [50]:
seasons[0]

,player_id,goals,assists,yellow_cards,red_cards,minutes_played,name,market_value_in_eur,age,height_in_cm,sub_position_encoded,position_encoded
0,1397.0,1.0,0.0,0.0,0.0,94.0,Michael Owen,1000000.0,33,173.0,3,0
1,3109.0,10.0,8.0,5.0,0.0,3042.0,Steven Gerrard,11500000.0,32,183.0,1,3
2,3118.0,0.0,1.0,1.0,0.0,196.0,Kieron Dyer,500000.0,34,170.0,9,3
4,3130.0,0.0,0.0,3.0,0.0,1181.0,Titus Bramble,2000000.0,31,192.0,2,1
6,3160.0,4.0,0.0,1.0,0.0,769.0,John Terry,12000000.0,32,187.0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...
274,128912.0,0.0,1.0,0.0,0.0,1015.0,Andre Wisdom,500000.0,19,186.0,11,1
286,134424.0,1.0,0.0,0.0,0.0,30.0,Nick Powell,5000000.0,18,183.0,0,3
287,134425.0,2.0,6.0,1.0,0.0,1658.0,Raheem Sterling,11500000.0,18,170.0,7,0
315,181579.0,0.0,1.0,0.0,0.0,403.0,James Ward-Prowse,500000.0,18,177.0,1,3


In [51]:
def value_over(name, option='age'):

    player_data = combined_stats_all[combined_stats_all['name'] == name]

    fig = px.line(player_data, x=option, y='market_value_in_eur', title=f'Market value of {name}', labels={'age' : 'Age', 'market_value_in_eur' : 'Market Value (EUR)'})



    fig.show()

In [52]:
value_over('Marcus Rashford')

In [53]:
merge_pos_val = players.merge(player_val, on='player_id')
merge_pos_val = merge_pos_val[(merge_pos_val['player_club_domestic_competition_id'].isin(top5_leagues))]
merge_pos_val['league'] = merge_pos_val['player_club_domestic_competition_id']
position_value = merge_pos_val.groupby(['sub_position', 'league']).agg({'market_value_in_eur_x' : 'mean'}).reset_index()

fig = px.bar(position_value, x='sub_position', y='market_value_in_eur_x', color='league', 
             barmode='group', 
             title='Average Value by Position in Major Leagues',
             labels={'market_value_in_eur_x': 'Average Market Value (EUR)', 'sub_position': 'Position', 'league': 'League'})

fig.show()


<h1>Próba predykcji różnymi modelami regresji</h1>

<h4>Random Forest dla sezonów oddzielnie dla piłkarzy top5 lig</h4>

In [54]:
total = 0
total_mae = 0

for i in range(len(seasons)):

    X = seasons_all_players[i].drop(columns=["market_value_in_eur", 'name', 'player_id'])
    y = seasons_all_players[i]['market_value_in_eur']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
    rf = RandomForestRegressor(random_state=42)
    rf.fit(X_train, y_train)

    y_pred = rf.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = rf.score(X_test, y_test)
    total += r2
    total_mae += mae
    print(f'Sezon {2012+i}/{2013+i}')
    print(f'MAE: {mae}')
    print(f'MSE: {mse}')
    print(f'R-squared: {r2}')
print(total/11)
print(total_mae/11)

Sezon 2012/2013
MAE: 4612949.468085106
MSE: 41911942125664.89
R-squared: 0.47014298963194623
Sezon 2013/2014
MAE: 5093848.333333333
MSE: 49646388080277.78
R-squared: 0.21179572969029914
Sezon 2014/2015
MAE: 5589271.276595744
MSE: 58035427343085.11
R-squared: 0.35844541428689936
Sezon 2015/2016
MAE: 6718985.148514852
MSE: 118241382918316.83
R-squared: 0.10071696649686679
Sezon 2016/2017
MAE: 5558540.404040404
MSE: 60359556191919.195
R-squared: 0.2073350737903764
Sezon 2017/2018
MAE: 9230722.222222222
MSE: 222218272103535.34
R-squared: 0.5486224845153829
Sezon 2018/2019
MAE: 10624639.175257731
MSE: 241363690721649.5
R-squared: 0.4505227214355665
Sezon 2019/2020
MAE: 13845831.578947369
MSE: 453405288410526.3
R-squared: 0.26320389692293544
Sezon 2020/2021
MAE: 11112543.26923077
MSE: 281136167838942.3
R-squared: 0.37384652299563104
Sezon 2021/2022
MAE: 10425207.070707072
MSE: 190576177992424.25
R-squared: 0.37562821340377905
Sezon 2022/2023
MAE: 10882342.857142856
MSE: 237623721776190.47
R-

<h4>Random Forest dla piłkarzy top5 lig</h4>

In [55]:


X = combined_stats_all.drop(columns=["market_value_in_eur", 'name', 'player_id'])
y = combined_stats_all['market_value_in_eur']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=1)
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = rf.score(X_test, y_test)
print(f'Sezon {2012+i}/{2013+i}')
print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R-squared: {r2}')

Sezon 2022/2023
MAE: 8631028.65497076
MSE: 192014629839785.06
R-squared: 0.3776858425926235


<h4>Ridge Regression dla piłkarzy top5 lig</h4>

In [56]:
X = combined_stats_all.drop(columns=["market_value_in_eur", 'name', 'player_id'])
y = combined_stats_all['market_value_in_eur']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=1)

param_grid = {'alpha': [0.1, 1, 10, 100, 1000]}
ridge = Ridge()
grid_search = GridSearchCV(ridge, param_grid, cv=5, scoring='r2')
grid_search.fit(X_train, y_train)


best_model = grid_search.best_estimator_


y_pred = best_model.predict(X_test)


mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Best alpha: {grid_search.best_params_}')
print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R-squared: {r2}')

Best alpha: {'alpha': 100}
MAE: 9131579.976752305
MSE: 207700956166509.3
R-squared: 0.3268468885036683


<h4>Linear Regression dla piłkarzy top5 lig</h4>

In [57]:
X = combined_stats_all.drop(columns=["market_value_in_eur", 'name', 'player_id'])
y = combined_stats_all['market_value_in_eur']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

lm = sm.OLS(y_train, X_train)
lm_fit = lm.fit()

y_pred = lm_fit.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)

print(f'Mean Absolute Error (MAE): {mae}')

lm_fit.summary()

Mean Absolute Error (MAE): 9126345.891769445


<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                            
===============================================================================
Dep. Variable:     market_value_in_eur   R-squared:                       0.297
Model:                             OLS   Adj. R-squared:                  0.296
Method:                  Least Squares   F-statistic:                     203.1
Date:                 Wed, 12 Jun 2024   Prob (F-statistic):          5.93e-323
Time:                         09:47:33   Log-Likelihood:                -77395.
No. Observations:                 4330   AIC:                         1.548e+05
Df Residuals:                     4320   BIC:                         1.549e+05
Df Model:                            9                                         
Covariance Type:             nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                 1.167e+07   6.31e+06      1.849      0.065   -7.07e+05    2.41e+07
goals                  1.64e+06   8.62e+04     19.021      0.000    1.47e+06    1.81e+06
assists               1.476e+06   1.28e+05     11.525      0.000    1.22e+06    1.73e+06
yellow_cards         -3.318e+05   1.12e+05     -2.973      0.003   -5.51e+05   -1.13e+05
red_cards             -7.81e+05    9.5e+05     -0.822      0.411   -2.64e+06    1.08e+06
minutes_played        2722.1975    321.953      8.455      0.000    2091.005    3353.390
age                  -6.358e+05    5.3e+04    -11.985      0.000    -7.4e+05   -5.32e+05
height_in_cm          4.815e+04   3.35e+04      1.439      0.150   -1.74e+04    1.14e+05
sub_position_encoded  1.045e+05    7.4e+04      1.412      0.158   -4.06e+04     2.5e+05
position_encoded       7.57e+05   2.17e+05      3.485      0.000    3.31e+05    1.18e+06
==============================================================================
Omnibus:                     2269.947   Durbin-Watson:                   2.018
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            23749.893
Skew:                           2.281   Prob(JB):                         0.00
Kurtosis:                      13.527   Cond. No.                     4.93e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.93e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

<h4>Próba klasyfikacji</h4>

In [58]:
# X = combined_stats_all.drop(columns=["market_value_in_eur", 'name', 'player_id'])
# y = combined_stats_all['market_value_in_eur']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=1)

# tree = DecisionTreeClassifier()

# path = tree.cost_complexity_pruning_path(X_train, y_train)

# alphas, impurities = path.ccp_alphas, path.impurities

# acc = list()

# for alpha in alphas:
#     t = DecisionTreeClassifier(ccp_alpha=alpha)
#     t.fit(X_train, y_train)
#     acc.append(t.score(X_val, y_val))

# plt.plot(alphas, acc)

<h4>Te same próby dla piłkarzy pochodzenia angielskiego z ligi angielskiej</h4>

<h4>Random Forest z podziałem na sezony</h4>

In [59]:
total = 0

for i in range(len(seasons)):

    X = seasons[i].drop(columns=["market_value_in_eur", 'name', 'player_id'])
    y = seasons[i]['market_value_in_eur']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
    rf = RandomForestRegressor(random_state=42)
    rf.fit(X_train, y_train)

    y_pred = rf.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = rf.score(X_test, y_test)
    total += r2
    print(f'Sezon {2012+i}/{2013+i}')
    print(f'MAE: {mae}')
    print(f'MSE: {mse}')
    print(f'R-squared: {r2}')

print(total/11)

Sezon 2012/2013
MAE: 4649907.407407408
MSE: 40980586805555.555
R-squared: -0.0522647240762919
Sezon 2013/2014
MAE: 4791250.0
MSE: 50316910336538.46
R-squared: 0.04764756739011666
Sezon 2014/2015
MAE: 5375192.307692308
MSE: 68779997596153.84
R-squared: -0.8127520907247845
Sezon 2015/2016
MAE: 6199400.0
MSE: 123454522500000.0
R-squared: 0.03564414189923881
Sezon 2016/2017
MAE: 3447944.4444444445
MSE: 22624910699074.074
R-squared: 0.2128353217181208
Sezon 2017/2018
MAE: 5265648.148148148
MSE: 37330923842592.59
R-squared: -0.27539614330152373
Sezon 2018/2019
MAE: 8980288.461538462
MSE: 179472231009615.38
R-squared: 0.2477994361125868
Sezon 2019/2020
MAE: 7154446.428571428
MSE: 94573469401785.72
R-squared: -0.7171970996290495
Sezon 2020/2021
MAE: 10553290.322580645
MSE: 253790105951612.9
R-squared: -0.5495309126225001
Sezon 2021/2022
MAE: 10874448.27586207
MSE: 229456947413793.1
R-squared: 0.25867327186077493
Sezon 2022/2023
MAE: 13122120.689655172
MSE: 327880247698275.9
R-squared: 0.263758

<h4>Random Forest bez podziału na sezony</h4>

In [60]:


X = combined_stats.drop(columns=["market_value_in_eur", 'name', 'player_id'])
y = combined_stats['market_value_in_eur']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = rf.score(X_test, y_test)
print(f'Sezon {2012+i}/{2013+i}')
print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R-squared: {r2}')

Sezon 2022/2023
MAE: 7846002.533783784
MSE: 154663494815667.25
R-squared: 0.38242029357385265


<h4>Ridge Regression</h4>

In [61]:
X = combined_stats.drop(columns=["market_value_in_eur", 'name', 'player_id'])
y = combined_stats['market_value_in_eur']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

param_grid = {'alpha': [0.1, 1, 10, 100, 1000]}
ridge = Ridge()
grid_search = GridSearchCV(ridge, param_grid, cv=5, scoring='r2')
grid_search.fit(X_train, y_train)


best_model = grid_search.best_estimator_


y_pred = best_model.predict(X_test)


mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Best alpha: {grid_search.best_params_}')
print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R-squared: {r2}')

Best alpha: {'alpha': 1000}
MAE: 8421456.819965566
MSE: 181558710743399.6
R-squared: 0.27502624058990166


<h4>Linear Regression</h4>

In [62]:
X = combined_stats.drop(columns=["market_value_in_eur", 'name', 'player_id'])
y = combined_stats['market_value_in_eur']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

lm = sm.OLS(y_train, X_train)
lm_fit = lm.fit()

y_pred = lm_fit.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)

print(f'Mean Absolute Error (MAE): {mae}')

lm_fit.summary()

Mean Absolute Error (MAE): 8447304.801343786


<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                            
===============================================================================
Dep. Variable:     market_value_in_eur   R-squared:                       0.279
Model:                             OLS   Adj. R-squared:                  0.274
Method:                  Least Squares   F-statistic:                     50.47
Date:                 Wed, 12 Jun 2024   Prob (F-statistic):           2.23e-77
Time:                         09:47:37   Log-Likelihood:                -21006.
No. Observations:                 1181   AIC:                         4.203e+04
Df Residuals:                     1171   BIC:                         4.208e+04
Df Model:                            9                                         
Covariance Type:             nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                 1.229e+07   1.11e+07      1.105      0.270   -9.54e+06    3.41e+07
goals                 1.389e+06   1.72e+05      8.076      0.000    1.05e+06    1.73e+06
assists               1.347e+06   2.33e+05      5.786      0.000     8.9e+05     1.8e+06
yellow_cards         -2.966e+05   1.99e+05     -1.491      0.136   -6.87e+05    9.37e+04
red_cards            -1.355e+06   1.72e+06     -0.789      0.430   -4.73e+06    2.01e+06
minutes_played        2544.3535    567.121      4.486      0.000    1431.667    3657.040
age                  -7.498e+05   8.42e+04     -8.902      0.000   -9.15e+05   -5.85e+05
height_in_cm          5.039e+04   5.85e+04      0.861      0.389   -6.44e+04    1.65e+05
sub_position_encoded  3.192e+05   1.32e+05      2.417      0.016    6.01e+04    5.78e+05
position_encoded      2.957e+05    4.1e+05      0.720      0.472    -5.1e+05     1.1e+06
==============================================================================
Omnibus:                      751.535   Durbin-Watson:                   1.982
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            10290.524
Skew:                           2.742   Prob(JB):                         0.00
Kurtosis:                      16.381   Cond. No.                     5.05e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.05e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

<h4>Dashboard</h4>

In [74]:
app = dash.Dash(__name__)


unique_seasons = list(season_stats.keys())

app.layout = html.Div(style={'backgroundColor' : 'black'},
    children=[
        html.H1(children='Football Transfer Market Dashboard', style={'color' : 'red'}),

        html.Label('Select Season', style={'color' : 'red'}),
        dcc.Dropdown(
            id='season-dropdown',
            options=[{'label': season, 'value': season} for season in unique_seasons],
            value=unique_seasons[0]
        ),
                
        html.Div(id='output-graph'),
        dcc.Input(id='player-name-input', type='text', placeholder='Wpisz nazwę gracza...'),
        html.Button('Potwierdź', id='confirm-button'),
        html.Div(id='value-over-output')
    ]
)

@app.callback(
    Output(component_id='output-graph', component_property='children'),
    Input(component_id='season-dropdown', component_property='value')
)
def update_graph(selected_season):
    season_data = season_stats[selected_season]
    top_players = season_data.nlargest(10, 'market_value_in_eur')

    fig = px.bar(top_players, 
                x='market_value_in_eur', 
                y='name', 
                orientation='h', 
                title=f'Top 10 Most Valuable Players in {selected_season}',
                labels={'market_value_in_eur': 'Market Value (in EUR)', 'name': 'Player'})
    return dcc.Graph(figure=fig)


fig_serie_a = go.Figure(data=[go.Pie(labels=club_value_serie_a['Club'], values=club_value_serie_a['Squad Market Value (EUR)'])])
fig_serie_a.update_layout(title='Market value of Serie A clubs')

fig_premier_league = go.Figure(data=[go.Pie(labels=club_value_premier_league['Club'], values=club_value_premier_league['Squad Market Value (EUR)'])])
fig_premier_league.update_layout(title='Market value of Premier League clubs')

fig_la_liga = go.Figure(data=[go.Pie(labels=club_value_la_liga['Club'], values=club_value_la_liga['Squad Market Value (EUR)'])])
fig_la_liga.update_layout(title='Market value of La Liga clubs')

fig_bundesliga = go.Figure(data=[go.Pie(labels=club_value_bundesliga['Club'], values=club_value_bundesliga['Squad Market Value (EUR)'])])
fig_bundesliga.update_layout(title='Market value of Bundesliga clubs')

fig_ligue = go.Figure(data=[go.Pie(labels=club_value_ligue['Club'], values=club_value_ligue['Squad Market Value (EUR)'])])
fig_ligue.update_layout(title='Market value of Ligue 1 clubs')

fig_positions = px.bar(position_value, x='sub_position', y='market_value_in_eur_x', color='league', 
             barmode='group', 
             title='Average Value by Position in Major Leagues',
             labels={'market_value_in_eur_x': 'Average Market Value (EUR)', 'sub_position': 'Position', 'league': 'League'})

data = []
for i, season in enumerate(seasons):
    player_data = season[season['name'] == 'Marcus Rashford']
    if not player_data.empty:
        season_data = {
            'season': f'Season {2012 + i}/{2013 + i}',
            'goals': player_data['goals'].values[0],
            'assists': player_data['assists'].values[0],
            'yellow_cards': player_data['yellow_cards'].values[0],
            'red_cards': player_data['red_cards'].values[0],
            'market_value_in_eur': player_data['market_value_in_eur'].values[0] / 1000000
        }
        data.append(season_data)
df = pd.DataFrame(data)
fig_stats = go.Figure()

fig_stats.add_trace(go.Bar(
    x=df['season'],
    y=df['goals'],
    name='Goals'
))

fig_stats.add_trace(go.Bar(
    x=df['season'],
    y=df['assists'],
    name='Assists'
))

fig_stats.add_trace(go.Bar(
    x=df['season'],
    y=df['yellow_cards'],
    name='Yellow Cards'
))

fig_stats.add_trace(go.Bar(
    x=df['season'],
    y=df['red_cards'],
    name='Red Cards'
))

fig_stats.add_trace(go.Bar(
    x=df['season'],
    y=df['market_value_in_eur'],
    name='Value'
))

fig_stats.update_layout(
    title='Statistics of Marcus Rashford Over Seasons',
    xaxis_title='Season',
    yaxis_title='Count',
    barmode='group',
    legend_title='Statistics'
)


app.layout.children.append(html.Div([
    dcc.Graph(figure=fig_serie_a),
    dcc.Graph(figure=fig_premier_league),
    dcc.Graph(figure=fig_la_liga),
    dcc.Graph(figure=fig_bundesliga),
    dcc.Graph(figure=fig_ligue),
    dcc.Graph(figure=fig_positions),
    dcc.Graph(figure=fig_stats),
    html.Div(id='value-over-input')
]))


@app.callback(
    Output('value-over-output', 'children'),
    Input('confirm-button', 'n_clicks'),
    State('player-name-input', 'value')
)


def update_value_over(n_clicks, player_name):
    if n_clicks is not None and player_name:
        result = value_over(player_name)
        return str(result)
    else:
        return "Proszę wpisać nazwę gracza i potwierdzić."


if __name__ == '__main__':
    app.run_server(debug=True)